In [1]:
using WordNet
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31


In [2]:
const WN_PATH="data/corpora/WordNet-2.1/"
db = DB(WN_PATH)

WordNet.DB

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [3]:
#ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model");
ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_300_i1.model");

In [4]:
function normal_probs(logprobs::Vector)
    ret = copy(logprobs)
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end
function weighted_average(logprobs, embeddings)
    ret = zeros(first(embeddings))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight.*embedding
    end
    ret
end

function all_logprobs_of_context(embed, context, middle_word)
    [Query.logprob_of_context(embed, context,wv; skip_oov=true, normalise_over_length=true)
        for wv in embed.embedding[middle_word]]
end

function synthesize_embedding(ee,context, middle_word)
    logprobs = all_logprobs_of_context(ee, context,middle_word)
    weighted_average(logprobs, ee.embedding[middle_word])
end

synthesize_embedding (generic function with 1 method)

In [5]:
function poormans_tokenize(source)
    cleaned = filter(s->!ispunct(s), lowercase(source))
    map(string,  split(cleaned))::Vector{SubString{String}}
end

function punctuation_space_tokenize(source)
    preprocced = lowercase(source)
    pass1=replace(preprocced,r"[[:punct:]]*[[:space:]][[:punct:]]*"," ")
    pass2=replace(pass1,r"[[:punct:]]*$|^[[:punct:]]*","")
    split(pass2)
end



  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


punctuation_space_tokenize (generic function with 1 method)

In [6]:
function load_sense_key_map(wn_basedir)
    path=joinpath(wn_basedir, "dict","index.sense")
    sense_key_map = Dict{Tuple{Int64,String},String}()
    for line in eachline(path)
        full_sense_key, offset_str, sense_num_str, tagcount_str = split(line) 
        lemma_name = first(split(full_sense_key,'%'))
        sense_offset = parse(Int64, offset_str)
        index = (sense_offset,lemma_name)
        @assert(!haskey(sense_key_map, index))
        sense_key_map[index] = full_sense_key
    end
    sense_key_map
end


function sense_key(lem::Lemma, ss::Synset,sense_key_map)
    sense_key_map[(ss.offset,lem.word)]
end
function sense_keys(lem::Lemma,sense_key_map)
    [sense_key_map[(ss_offset,lem.word)] for ss_offset in lem.synset_offsets ]
end

sense_keys (generic function with 1 method)

In [7]:
sense_key_map = load_sense_key_map(WN_PATH);

In [9]:
sense_key_map[2134693,"discover"]

"discover%2:39:03::"

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [10]:
using LightXML

In [11]:
immutable WsdChallenge
    id::String
    word::String
    lemma::String
    pos::Char
    context::Vector{String}
end

function load_challenges(xml_file="data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml")
    xdoc = parse_file(xml_file)
    xroot = root(xdoc)
    Task() do
        for text_node in child_elements(xroot)
            #@show text_node
            #text = child_nodes(text_node) |> collect
            #println(text)
            for sentence_node in child_elements(text_node)
                sentence = punctuation_space_tokenize(content(sentence_node))

                for lemma_node in child_elements(sentence_node)
                    word = content(lemma_node) |> lowercase
                    lemma = attribute(lemma_node,"lemma")|> lowercase
                    pos = first(attribute(lemma_node,"pos"))
                    id = attribute(lemma_node,"id")
                    context = filter(w->w!=word, sentence)
                    produce(WsdChallenge(id, word, lemma, pos, context))
                end
            end
        end
    end
end
challenges = load_challenges() |> collect;

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in bytestring(::Ptr{UInt8}) at ./deprecated.jl:50
 in _xcopystr at /home/ubuntu/.julia/v0.5/LightXML/src/LightXML.jl:0 [inlined]
 in content(::LightXML.XMLNode) at /home/ubuntu/.julia/v0.5/LightXML/src/nodes.jl:172
 in (::##3#5{LightXML.XMLElement})() at ./In[11]:18
while loading In[11], in expression starting on line 32
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in bytestring(::Ptr{UInt8}) at ./deprecated.jl:50
 in _xcopystr at /home/ubuntu/.julia/v0.5/LightXML/src/LightXML.jl:0 [inlined]
 in #attribute#1(::Bool, ::Function, ::LightXML.XMLElement, ::String) at /home/ubuntu/.julia/v0.5/LightXML/src/nodes.jl:227
 in (::##3#5{LightXML.XMLElement})() at ./In[11]:22
while loading In[11], in expression starting on line 32


In [12]:
immutable WsdSolution
    id
    lemma
    pos
    solutions
end
only_of_pos(data, pos) = filter(d->d.pos==pos, data)

function load_solutions(key_file="data/corpora/wsd/semeval2007_t7/key/dataset21.test.key")
    map(eachline(key_file)) do line
        line_data, comment = split(line,"!!")
        fields = split(line_data)
        doc_id = fields[1]
        instance_id = fields[2]
        solutions = fields[3:end]
        
        lemma,pos = match(r"lemma=(.*)#(.)", comment).captures
        WsdSolution(instance_id, lemma, pos[1], solutions)
    end
end

solutions = load_solutions();

In [13]:
using Query

In [14]:
challenge = challenges[1]

WsdChallenge("d001.s001.t001","editorial","editorial",'n',String["your","oct","6","the","ill","homeless","referred","to","research","by","us","and","six","of","our","colleagues","that","was","reported","in","the","sept","8","issue","of","the","journal","of","the","american","medical","association"])

In [15]:
function sense_frequency(ss::Synset, lem::Lemma)
    lem_count = lem.tagsense_count
    lem_count+0.1(sum(values(ss.word_counts))-lem_count)
end

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


sense_frequency (generic function with 1 method)

In [16]:
lexically_informed_embeddings(db,ee, lemma_word, pos) = lexically_informed_embeddings(db,ee, lemma_word, lemma_word, pos)

function lexically_informed_embeddings(db,ee, word,lemma_word, pos)
    source_wvs = ee.embedding[word]
    lemma = db[pos, lemma_word]
    target_synsets::Vector{Synset} = synsets(db, lemma)
    if length(source_wvs) == 1
        #Use MCWS
        score, index = findmax(sense_frequency(ss,lemma) for ss in target_synsets)
        target_synsets=[target_synsets[index]] # Throw out others
    end
            
    embeddings = Vector{eltype(source_wvs)}(length(target_synsets))
    for (ii,synset) in enumerate(target_synsets)
        context::Vector{SubString{String}} = collect(filter!(w->w!=word, punctuation_space_tokenize(synset.gloss)))
        embeddings[ii] = synthesize_embedding(ee,context, word)
    end
        
    embeddings,target_synsets,lemma
end

lexically_informed_embeddings (generic function with 2 methods)

In [17]:
methods(logprob_of_context)

# 2 methods for generic function "logprob_of_context":
logprob_of_context{S<:String}(embed::WordEmbeddings.WordEmbedding, context::AbstractArray{S,1}, middle_word::S; kwargs...) at /mnt/phd/prototypes/SenseSplittingWord2Vec/src/Query.jl:142
logprob_of_context(embed::WordEmbeddings.GenWordEmbedding, context, input::Array{Float32,1}; skip_oov, normalise_over_length) at /mnt/phd/prototypes/SenseSplittingWord2Vec/src/Query.jl:148

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [18]:
function supervised_wsd(challenge::WsdChallenge, ee::WordSenseEmbedding, wn::DB, sense_key_map)
    try
        embeddings,target_synsets,lemma = lexically_informed_embeddings(wn,ee,
                                                            challenge.word, 
                                                            challenge.lemma, challenge.pos)
        #sense_index = Query.WSD(ee, embeddings, challenge.context; skip_oov=true)
        logprobs_of_context = [logprob_of_context(ee,challenge.context, wv; skip_oov=true, normalise_over_length=true) 
                                    for wv in embeddings] 
        
        sense_freqs = 1# Float32[sense_frequency(ss,lemma)+1 for ss in target_synsets]
        sense_freqs./=sum(sense_freqs)
        
        sense_index= indmax(log(sense_freqs).+logprobs_of_context)

        
        synset = target_synsets[sense_index]
        sk = sense_key(lemma, synset, sense_key_map)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end

supervised_wsd (generic function with 1 method)

In [19]:
function evalute_on_wsd_challenge(challenges, solutions, ee::WordSenseEmbedding, wn::DB, sense_key_map)
    correct = 0
    incorrect = 0
    notattempted = 0
    for (challenge, ground_solution) in zip(challenges, solutions)
        assert(challenge.id == ground_solution.id)
        output_sense = supervised_wsd(challenge, ee,wn,sense_key_map)
        if isnull(output_sense)
            notattempted+=1
        else
            if get(output_sense) ∈ ground_solution.solutions
                correct+=1
            else
                incorrect+=1
            end
        end
    end
    precision = correct/(correct+incorrect)
    recall = correct/(correct+incorrect+notattempted)
    f1 = (2*precision*recall) / (precision+recall)
    return precision, recall, f1
end

evalute_on_wsd_challenge (generic function with 1 method)

In [20]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, ee, db::DB, sense_key_map))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), ee, db::DB, sense_key_map))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), ee, db::DB, sense_key_map))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), ee, db::DB, sense_key_map))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), ee, db::DB, sense_key_map))


overall: 		(0.6311437758383096,0.6055531070956368,0.6180836707152496)
only nouns  : 		(0.6330532212885154,0.6119133574007221,0.6223038090867371)
only verbs  : 		(0.5597147950089126,0.5313028764805414,0.5451388888888888)
only adjecti: 		(0.6704545454545454,0.6519337016574586,0.6610644257703082)
only adverbs: 		(0.7564766839378239,0.7019230769230769,0.7281795511221945)


### 201004_100_i1
overall: 		(0.6113699906803355,0.5782282944028206,0.5943374858437146)
only nouns  : 		(0.6168401135288553,0.5884476534296029,0.6023094688221708)
only verbs  : 		(0.5298372513562387,0.4957698815566836,0.5122377622377623)
only adjecti: 		(0.6618075801749271,0.6270718232044199,0.6439716312056738)
only adverbs: 		(0.7253886010362695,0.6730769230769231,0.6982543640897756)

### 201004_300_i1
overall: 		(0.6311437758383096,0.6055531070956368,0.6180836707152496)
only nouns  : 		(0.6330532212885154,0.6119133574007221,0.6223038090867371)
only verbs  : 		(0.5597147950089126,0.5313028764805414,0.5451388888888888)
only adjecti: 		(0.6704545454545454,0.6519337016574586,0.6610644257703082)
only adverbs: 		(0.7564766839378239,0.7019230769230769,0.7281795511221945)


In [ ]:
println("only nouns: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), ee, db::DB, sense_key_map))


In [23]:
sum((collect(values(ee.distribution)).*ee.corpus_size).>1000)

36529

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [ ]:
println("Foo")
f(y)=(x->2x)(y);
println("This is never displayed")